In [1]:
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os, sys

sys.path.append('../utilities')
from alignFaces import *

In [2]:
data_params = {
    "dataset":"../../facial_recognition_data/faces_dataset/", # path to input directory of faces and images
    "embeddings":"../../output/embeddings.pickle", # path to output of database of facial embeddings
    "detector":"../../facial_detection/models/", # path to OpenCV's deep learning face detector
    "embedding_model":"../models/openface.nn4.small2.v1.t7", # path to OpenCV's deep learning face embedding model
    "confidence":0.7 # min probability to filter weak detections
}

In [3]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join([data_params["detector"], "deploy.prototxt"])
modelPath = os.path.sep.join([data_params["detector"], "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(data_params["embedding_model"])

[INFO] loading face detector...
[INFO] loading face recognizer...


In [4]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(paths.list_images(data_params["dataset"]))
print(*imagePaths, sep="\n")
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

[INFO] quantifying faces...
../../facial_recognition_data/faces_dataset/andrew\IMG_1247.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1248.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1249.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1250.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1251.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1252.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1253.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1254.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1255.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1256.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1257.jpg
../../facial_recognition_data/faces_dataset/andrew\IMG_1258.jpg
../../facial_recognition_data/faces_dataset/andrew\WIN_20220217_15_22_57_Pro.jpg
../../facial_recognition_data/faces_dataset/andrew\WIN_20220217_15_22_58_Pro.jpg
../../facial_recognition_data/faces_datase

In [5]:
# loop over the image paths
af = AlignFaces()
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
    # The right side of the last occurance of a '/' character is gotten from the path.
    # Then the string is split based on the '\' character which prepends the image name.
    # The left side of the split, representing the directory/person name is saved as name.
    name = imagePath[imagePath.rindex('/')+1:].split('\\')[0]
    image = cv2.imread(imagePath) # Load the image
    image = af.align_faces(image, imagePath) # Align the image
    image = imutils.resize(image, width=600) # Resize the image (while maintaining aspect ratio)
    (h, w) = image.shape[:2] # Grab the image dimensions.
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    
    # ensure at least one face was found
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]

        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        if confidence > data_params["confidence"]:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI and grab the ROI dimensions
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue
            
            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            # add the name of the person + corresponding face
            # embedding to their respective lists
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
            total += 1

[INFO] processing image 1/94
[INFO] processing image 2/94
[INFO] processing image 3/94
[INFO] processing image 4/94
[INFO] processing image 5/94
[INFO] processing image 6/94
[INFO] processing image 7/94
[INFO] processing image 8/94
[INFO] processing image 9/94
[INFO] processing image 10/94
[INFO] processing image 11/94
[INFO] processing image 12/94
[INFO] processing image 13/94
[INFO] processing image 14/94
[INFO] processing image 15/94
[INFO] processing image 16/94
[INFO] processing image 17/94
[INFO] processing image 18/94
[INFO] processing image 19/94
[INFO] processing image 20/94
[INFO] processing image 21/94
[INFO] processing image 22/94
[INFO] processing image 23/94
[INFO] processing image 24/94
[INFO] processing image 25/94
[INFO] processing image 26/94
[INFO] processing image 27/94
[INFO] processing image 28/94
[INFO] processing image 29/94
[INFO] processing image 30/94
[INFO] processing image 31/94
[INFO] processing image 32/94
[INFO] processing image 33/94
[INFO] processing i

In [6]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(data_params["embeddings"], "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 91 encodings...
